## 補足　3クラス潜在変数モデル

<a href="https://colab.research.google.com/github/makaishi2/python_bayes_intro/blob/main/sample-notebooks/A_3%E3%82%AF%E3%83%A9%E3%82%B9%E6%BD%9C%E5%9C%A8%E5%A4%89%E6%95%B0%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### 共通処理

In [ ]:
%matplotlib inline
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np

# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt

# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib

# pandas用ライブラリ
import pandas as pd

# データフレーム表示用関数
from IPython.display import display

# seaborn
import seaborn as sns

# 表示オプション調整

# NumPy表示形式の設定
np.set_printoptions(precision=3, floatmode='fixed')

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 方眼表示ON
plt.rcParams['axes.grid'] = True

# データフレームでの表示精度
pd.options.display.float_format = '{:.3f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

In [ ]:
import pymc as pm
import arviz as az

print(f"Running on PyMC v{pm.__version__}")
print(f"Running on ArViz v{az.__version__}")

### A.1 カテゴリカル分布

#### 確率モデル定義

In [ ]:
# パラメータ設定
p = [0.2, 0.5, 0.3]

model1 = pm.Model()
with model1:
    # pm.Categorical: カテゴリカル分布
    # p: 各要素の発生確率
    x = pm.Categorical('x', p=p)

#### 事前分布のサンプリングとサンプル値抽出

In [ ]:
with model1:
    # 事前分布のサンプリング
    prior_samples1 = pm.sample_prior_predictive(random_seed=42)

x_samples1 = prior_samples1['prior']['x'].values
print(x_samples1)

#### サンプリング結果の可視化

In [ ]:
ax = az.plot_dist(x_samples1)
ax.set_title(f'カテゴリカル分布　p={p}');

### A.2 ディリクレ分布

#### 確率モデル定義

In [ ]:
# パラメータ設定
n_components = 3

model2 = pm.Model()
with model2:
    # ディリクレ分布
    # a:パラメータ　[1, 1, 1]だと一様分布
    p = pm.Dirichlet('p', a=np.ones(n_components))

#### 事前分布のサンプリングとサンプル値抽出

In [ ]:
with model2:
    # サンプル値取得
    samples2 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples2 = samples2['prior']['p'].values
# 桁数が多いので先頭10個だけに限定
print(x_samples2[:,:10])

#### サンプリング結果の可視化

In [ ]:
# サンプル値の可視化
samples2 = x_samples2.reshape(-1,3)
plt.title('ディリクレ分布 a=(1,1,1)の場合')
x1 = samples2[:,0]
x2 = samples2[:,1]
plt.scatter(x1,x2, s=5);

### A.3 3クラス潜在変数モデル

#### データ読み込み

In [ ]:
# アイリスデータセットの読み込み
df = sns.load_dataset('iris')

# 先頭5行の確認
display(df.head())

#  speciesの分布確認
df['species'].value_counts()

#### 変数設定

In [ ]:
#  観測値データ
X = df['petal_width'].values

#  データ件数
N = X.shape

# 分類先クラス数
n_components = 3

#### 確率モデル定義

In [ ]:
model3 = pm.Model()

with model3:
    #  観測値をpm.ConstantDataで定義する
    X_data = pm.ConstantData('X_data', X)

    # p:  それぞれの値を取るの確率を示す3要素のベクトル
    p = pm.Dirichlet('p', a=np.ones(n_components))

    # s: pの確率値を基に0, 1, 2のいずれかの値を返す
    s = pm.Categorical('s', p=p, shape=N)

    # mus: 3つの花の種類毎の平均値
    mus = pm.Normal('mus', mu=0.0, sigma=10.0, shape=n_components)

    # taus: 3つの花の種類毎のバラツキ
    # 標準偏差sigmasとは　taus = 1/(sigmas*sigmas) の関係にある
    taus = pm.HalfNormal('taus', sigma=10.0, shape=n_components)

    # グラフ描画など分析でsigmaが必要なため、tauからsigmaを求めておく
    sigmas = pm.Deterministic('sigmas', 1/pm.math.sqrt(taus))

    # 各観測値ごとに潜在変数からmuとtauを求める
    mu = pm.Deterministic('mu', mus[s])
    tau = pm.Deterministic('tau', taus[s])

    # 正規分布に従う確率変数X_obsの定義
    X_obs = pm.Normal('X_obs', mu=mu, tau=tau, observed=X_data)

# モデル構造可視化
g = pm.model_to_graphviz(model3)
display(g);

#### サンプリング

In [ ]:
with model3:
    idata3 = pm.sample(
      chains=1, draws=2000, target_accept=0.99,
      random_seed=42)

#### 推論結果の確認

In [ ]:
az.plot_trace(idata3, var_names=['p', 'mus', 'sigmas'], compact=False)
plt.tight_layout();

#### 統計処理の集計

In [ ]:
summary3 = az.summary(idata3, var_names=['p', 'mus', 'sigmas'])
display(summary3)

#### ヒストグラムと推論結果の重ね描き

In [ ]:
# 正規分布関数の定義
def norm(x, mu, sigma):
    return np.exp(-((x - mu)/sigma)**2/2) / (np.sqrt(2 * np.pi) * sigma)

# 推論結果から各パラメータの平均値を取得
mean3 = summary3['mean']

# muの平均値取得
mean3_mu0 = mean3['mus[0]']
mean3_mu1 = mean3['mus[1]']
mean3_mu2 = mean3['mus[2]']

# sigmaの平均値取得
mean3_sigma0 = mean3['sigmas[0]']
mean3_sigma1 = mean3['sigmas[1]']
mean3_sigma2 = mean3['sigmas[2]']

# グラフ描画
x = np.arange(0.0, 3.0, 0.05)
plt.rcParams['figure.figsize']=(8,6)
fig, ax = plt.subplots()
sns.histplot(
    data=df,
    bins=np.arange(0.0, 3.0, 0.1),
    x='petal_width',
    hue='species', kde=True)
plt.setp(ax.get_xticklabels(), rotation=90)
plt.title('petal_widthのヒストグラム')
plt.xticks(np.arange(0.0, 3.0, 0.1));
plt.title('ヒストグラムと正規分布関数の重ね描き')
plt.plot(x, norm(x, mean3_mu0, mean3_sigma0)*5.0, c='y', lw=3)
plt.plot(x, norm(x, mean3_mu1, mean3_sigma1)*5.0, c='g', lw=3)
plt.plot(x, norm(x, mean3_mu2, mean3_sigma2)*5.0, c='b', lw=3);

### A.4 3クラス潜在変数モデル(失敗例)

#### 確率モデル定義

In [ ]:
model4 = pm.Model()

with model4:
    #  観測値をpm.ConstantDataで定義する
    X_data = pm.ConstantData('X_data', X)

    # p:  それぞれの値を取るの確率を示す3要素のベクトル
    p = pm.Dirichlet('p', a=np.ones(n_components))

    # s: pの確率値を基に0, 1, 2のいずれかの値を返す
    s = pm.Categorical('s', p=p, shape=N)

    # mus: 3つの花の種類毎の平均値
    mus = pm.Normal('mus', mu=0.0, sigma=10.0, shape=n_components)

    # taus: 3つの花の種類毎のバラツキ
    # 標準偏差sigmasとは　taus = 1/(sigmas*sigmas) の関係にある
    sigmas = pm.HalfNormal('sigmas', sigma=10.0, shape=n_components)

    # 各観測値ごとに潜在変数からmuとtauを求める
    mu = pm.Deterministic('mu', mus[s])
    sigma = pm.Deterministic('sigma', sigmas[s])

    # mu[s], tau[s]: 潜在変数による参照
    X_obs = pm.Normal('X_obs', mu=mu, sigma=sigma, observed=X_data)

# モデル構造可視化
g = pm.model_to_graphviz(model4)
display(g);

#### サンプリングと推論結果の確認

In [ ]:
with model4:
    idata4 = pm.sample(
      chains=1, draws=2000, target_accept=0.99,
      random_seed=42)

az.plot_trace(idata4, var_names=['p', 'mus', 'sigmas'], compact=False)
plt.tight_layout();

### A.5 3クラス潜在モデル(改良版)

#### 確率モデル定義

In [ ]:
model5 = pm.Model()

with model5:
    #  観測値をpm.ConstantDataで定義する
    X_data = pm.ConstantData('X_data', X)

    # p:  それぞれの値を取るの確率を示す3要素のベクトル
    p = pm.Dirichlet('p', a=np.ones(n_components))

    # s: pの確率値を基に0, 1, 2のいずれかの値を返す
    s = pm.Categorical('s', p=p, shape=N)

    # mus: 3つの花の種類毎の平均値
    mu0 = pm.HalfNormal('mu0', sigma=10.0)
    delta0 = pm.HalfNormal('delta0', sigma=10.0)
    mu1 = pm.Deterministic('mu1', mu0+delta0)
    delta1 = pm.HalfNormal('delta1', sigma=10.0)
    mu2 = pm.Deterministic('mu2', mu1+delta1)
    mus = pm.Deterministic('mus', pm.math.stack([mu0, mu1, mu2]))

    # taus: 3つの花の種類毎のバラツキ
    # 標準偏差sigmasとは　taus = 1/(sigmas*sigmas) の関係にある
    taus = pm.HalfNormal('taus', sigma=10.0, shape=n_components)

    # グラフ描画など分析でsigmaが必要なため、tauからsigmaを求めておく
    sigmas = pm.Deterministic('sigmas', 1/pm.math.sqrt(taus))

    # 各観測値ごとに潜在変数からmuとtauを求める
    mu = pm.Deterministic('mu', mus[s])
    tau = pm.Deterministic('tau', taus[s])

    # mu[s], tau[s]: 潜在変数による参照
    X_obs = pm.Normal('X_obs', mu=mu, tau=tau, observed=X_data)

# モデル構造可視化
g = pm.model_to_graphviz(model5)
display(g);

#### サンプリングと推計結果の確認

In [ ]:
with model5:
    idata5 = pm.sample(target_accept=0.99, random_seed=42)

plt.rcParams['figure.figsize']=(6,6)
az.plot_trace(idata5, var_names=['p', 'mus', 'sigmas'], compact=False)
plt.tight_layout();

#### 統計処理の集計

In [ ]:
summary5 = az.summary(idata5, var_names=['p', 'mus', 'sigmas'])
display(summary5)

#### ヒストグラムと推論結果の重ね描き

In [ ]:
# 推論結果から各パラメータの平均値を取得
mean5 = summary5['mean']

# muの平均値取得
mean5_mu0 = mean5['mus[0]']
mean5_mu1 = mean5['mus[1]']
mean5_mu2 = mean5['mus[2]']

# sigmaの平均値取得
mean5_sigma0 = mean5['sigmas[0]']
mean5_sigma1 = mean5['sigmas[1]']
mean5_sigma2 = mean5['sigmas[2]']

# グラフ描画
x = np.arange(0.0, 3.0, 0.05)
plt.rcParams['figure.figsize']=(8,6)
fig, ax = plt.subplots()
sns.histplot(
    data=df,
    bins=np.arange(0.0, 3.0, 0.1),
    x='petal_width',
    hue='species', kde=True)
plt.setp(ax.get_xticklabels(), rotation=90)
plt.title('petal_widthのヒストグラム')
plt.xticks(np.arange(0.0, 3.0, 0.1));
plt.title('ヒストグラムと正規分布関数の重ね描き')
plt.plot(x, norm(x, mean5_mu0, mean5_sigma0)*5.0, c='b', lw=3)
plt.plot(x, norm(x, mean5_mu1, mean5_sigma1)*5.0, c='y', lw=3)
plt.plot(x, norm(x, mean5_mu2, mean5_sigma2)*5.0, c='g', lw=3);